<a href="https://colab.research.google.com/github/Jaesu26/Dacon-Basic/blob/main/%EC%88%98%ED%99%94%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수화 이미지 분류 경진대회

## 패키지 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import re
import os
import warnings
from tqdm import tqdm
from glob import glob
from sklearn.model_selection import StratifiedKFold, train_test_split

warnings.filterwarnings(action = 'ignore')

In [46]:
SEED = 22
N_FOLD = 10
SAVE_PATH = './weight'
LEARNING_RATE = 0.005
EPOCHS = 300
BATCH_SIZE = 32

In [3]:
from google.colab import drive

drive.mount('/content/drive') ## 구글코랩과 구글드라이브 연결

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd '/content/drive/MyDrive/Dacon-Basic/수화이미지-분류/Data'

/content/drive/MyDrive/Dacon-Basic/수화이미지-분류/Data


In [5]:
# !unzip -o '/content/drive/MyDrive/Dacon-Basic/수화이미지-분류/Data/user_data.zip의 사본'

In [6]:
%cd '/content/drive/MyDrive/Dacon-Basic/수화이미지-분류'

/content/drive/MyDrive/Dacon-Basic/수화이미지-분류


In [7]:
df = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')
submission = pd.read_csv('./Data/sample_submission.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  858 non-null    object
 1   label      858 non-null    object
dtypes: object(2)
memory usage: 13.5+ KB


`-` 결측치는 없다

In [9]:
df.head()

,file_name,label
0,001.png,10-2
1,002.png,10-1
2,003.png,3
3,004.png,8
4,005.png,9


`-` 파일 이름과 라벨명이 한 쌍으로 되어있음

## 데이터 셋 및 딥러닝 모델 정의

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms, models
import gc

In [11]:
## accuracy 계산
def accuracy(true, pred):
    return sum(true == pred) / len(true)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
print(f'현재 device는 {device}입니다')

현재 device는 cuda입니다


In [14]:
def seed_everything(seed: int = 22):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [15]:
seed_everything(SEED) ## 재현을 위한 seed 고정

- 학습 이미지와 이미지 경로

In [16]:
def load_images(path):
    images = []
    for img in sorted(glob(path + '/*.png')): ## path에 들어있는 png 파일을 검색(1번부터 차례대로 검색해야 된다)
        an_img = PIL.Image.open(img)  
        img_array = np.array(an_img)  
        images.append(img_array)  
            
    images = np.array(images)
    return images

In [17]:
train_images = load_images(path='./Data/train')
test_images = load_images(path='./Data/test')

In [18]:
train_img_path_list = sorted(glob('./Data/train/*.png'))
test_img_path_list = sorted(glob('./Data/test/*.png'))

- 데이터 셋과 데이터 로더

In [19]:
class CustomDataset(Dataset):
  
    def __init__(self, images: np.array, label_list, train_mode=True, transforms=None):
        self.images = images
        self.label_list = label_list
        self.transforms = transforms
        self.train_mode = train_mode
    
    def __getitem__(self, idx):
        image = self.images[idx]    
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[idx]
            return image, label
        
        return image ## test는 라벨이 없다
    
    def __len__(self):
        return len(self.images) ## 데이터 개수 반환

- 데이터 변환

In [20]:
RGB_MEAN = [np.mean((train_images[..., i] / 255)) for i in range(3)] ## 255로 나눈후 RGB 각각의 평균
RGB_STD = [np.std(train_images[..., i] / 255) for i in range(3)]  ## 255로 나눈후 RGB 각각의 표준편차

In [64]:
## 데이터 변환 방식을 정의(Agumentation)
def get_transform(train_mode=True):
    trans = transforms.Compose([
        transforms.ToPILImage(), ## numpy array(H X W X C)를 PIL이미지로 바꾼다
        transforms.Resize([128, 128]),
        transforms.ToTensor(), ## PIL or numpy array(H X W X C; 0 ~ 255) -> torch tensor(H X W X C; 0 ~ 1)
        transforms.Normalize(RGB_MEAN, RGB_STD) ## 평균을 0, 표준편차를 1로 만듦
    ])
    
    if train_mode:
        trans = transforms.Compose([
            transforms.ToPILImage(), ## numpy array(H X W X C)를 PIL이미지로 바꾼다
            transforms.CenterCrop(size=196), ## size X size 크기의 이미지를 중앙에서 crop
            transforms.Resize([128, 128]), ## 이미지 사이즈 변경
            # transforms.transforms.GaussianBlur(kernel_size=(5, 5)),
            transforms.RandomHorizontalFlip(p=0.5), ## 50%의 확률로 수평으로 뒤집음
            # transforms.RandomRotation(degrees=10), ## 무작위로 -degrees~degrees 각도 회전
            transforms.ToTensor(), ## PIL or numpy array(H X W X C; 0 ~ 255) -> torch tensor(H X W X C; 0 ~ 1)
            transforms.Normalize(mean=RGB_MEAN, std=RGB_STD) ## 평균을 0, 표준편차를 1로 만듦
        ])

    return trans ## 데이터 변환 방식을 반환

`-` 라벨을 0~10로 변환하겠다

In [22]:
def get_label_map() -> dict:
    label_map = dict()
    label_map['10-1'] = 10
    label_map['10-2'] = 0
    for i in range(1, 10):
        label_map[str(i)] = i

    return label_map

In [23]:
def label_encoding_transform(label: pd.Series) -> pd.Series:
    label_map = get_label_map()
    encoded_label = label.apply(lambda x: label_map[x])
    return encoded_label

In [24]:
def label_encoding_inverse_transform(encoded_label: pd.Series) -> pd.Series:
    label_map = get_label_map()
    label_inverse_map = dict(zip(label_map.values(), label_map.keys()))
    label = encoded_label.apply(lambda x: label_inverse_map[x])
    return label

In [25]:
target = label_encoding_transform(df['label'])

- CNN 신경망

In [66]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.cnn_model = torch.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=1, padding='same'), ## padding='same' 옵션을 사용할려면 stride가 1이어야 한다
            nn.ELU(),
            nn.BatchNorm2d(32),
  
            nn.Conv2d(32, 16, kernel_size=(3, 3), stride=1, padding='same'), 
            nn.ELU(), 
            nn.BatchNorm2d(16),

            nn.Conv2d(16, 16, kernel_size=(3, 3), stride=1, padding='same'), 
            nn.ELU(), 
            nn.BatchNorm2d(16),

            nn.MaxPool2d(kernel_size=(3, 3), stride=2), ## max pooling

            nn.Conv2d(16, 16, kernel_size=(3, 3), stride=1, padding='same'), 
            nn.ELU(), 
            nn.BatchNorm2d(16),

            nn.Conv2d(16, 16, kernel_size=(3, 3), stride=1, padding='same'), 
            nn.ELU(), 
            nn.BatchNorm2d(16),

            nn.Conv2d(16, 16, kernel_size=(3, 3), stride=1, padding='same'), 
            nn.ELU(), 
            nn.BatchNorm2d(16),

            nn.MaxPool2d(kernel_size=(3, 3), stride=2), ## max pooling

            nn.AdaptiveAvgPool2d(1) ## Flatten역할 ## 이미지를 평균내서 1*1 크기로 만든다
        )

        self.linear_model = nn.Sequential(
            nn.Linear(16, 64),
            nn.ELU(),
            nn.Dropout(0.3),
            nn.Linear(64, 11) ## softmax는 옵티마이저(CrossEntorpyLoss)에서 수행
        )
      
    def forward(self, x):
        x = self.cnn_model(x)
        x = x.squeeze()
        x = self.linear_model(x)
        return x

- Linear layer 가중치 초기화

In [28]:
## Linear layer 가중치 초기화
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        y = m.in_features
        m.weight.data.normal_(0.0, 1/np.sqrt(y))
        m.bias.data.fill_(0)

- 조기 중단

In [29]:
class EarlyStopping:
    ## 코드 참고: https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
    
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='./weight', n_fold=1):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 에폭
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint 저장 경로
                            Default: 'checkpoint.pt'
            n_fold (int): 현재 학습을 진행하는 fold의 순서
                            Default: 1
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.val_acc = None
        self.delta = delta
        self.path = path
        self.n_fold = n_fold

    def __call__(self, model, val_loss, val_acc):

        score = -val_loss ## val_loss는 작을수록 좋다 ## score는 0에 가까울수록 좋다

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, val_acc, model)  
        elif score < self.best_score + self.delta: ## loss가 개선되지 않았을 때
            self.counter += 1 ## 카운팅 +1
            # print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience: ## 만약 loss가 개선되지 않은 스탭이 patience보다 크거나 같아진다면 조기중단
                self.early_stop = True
        else: ## loss가 개선됨
            self.best_score = score ## score 갱신
            self.save_checkpoint(val_loss, val_acc, model) ## loss와 model 저장
            self.counter = 0 ## loss가 개선되었으므로 0으로 초기화

    def save_checkpoint(self, val_loss, val_acc, model):
        """validation loss가 감소하면 모델을 저장"""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.5f} -> {val_loss:.5f})  Saving model ...')
        torch.save(model.state_dict(), self.path + f'/best_{self.n_fold}.pt') ## 모델의 계층별 가중치를 지정한 경로에 저장
        self.val_loss_min = val_loss ## 모델이 더 좋게 갱신되었으므로 이때의 valid loss를 기준치로 변경
        self.val_acc = val_acc ## 이때의 valid accuracy도 변경해준다

## 모델 교차검증

In [30]:
skfold = StratifiedKFold(n_splits=N_FOLD, random_state=SEED, shuffle=True) ## k겹 교차검증

In [31]:
loss_fn = torch.nn.CrossEntropyLoss()   ## 손실 함수에 소프트맥스 함수 포함 -> net 내부에서 마지막 활성화함수로 소프트맥스 사용안해도 됨

In [32]:
def train(model: nn.Module, dataloader, optimizer, loss_fn, scheduler=None):
    """dataloader의 데이터를 사용하여 학습된 모델과 평균 훈련 오차를 반환"""
    model.train() ## 훈련모드
    train_avg_loss = 0 ## 에폭별 배치단위 평균 훈련 오차
    train_total_batch = len(dataloader) ## 배치 크기

    for X, y in dataloader: ## 미니 배치 단위로 꺼내온다, X는 미니 배치, y는 레이블
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad() ## 그래디언트 초기화
        yhat = model(X) ## y_hat을 구한다
        loss = loss_fn(yhat, y).to(device) ## 오차를 계산 ## train loss
        loss.backward()  ## 미분
        optimizer.step() ## 업데이트
        train_avg_loss += (loss.item() / train_total_batch) ## 각 배치마다 훈련 오차 누적

    ## epoch마다 학습률 조절
    if scheduler is not None:
        scheduler.step()

    return model, train_avg_loss

In [33]:
def evaluate(model: nn.Module, dataloader, loss_fn):
    """dataloader의 데이터를 사용하여 모델에 대한 평균 평가 오차와 평가 정확도를 반환"""
    valid_avg_acc, valid_avg_loss = 0, 0

    model.eval() ## 평가모드
    with torch.no_grad(): ## 평가할 땐 역전파를 쓸 필요가 없으니까
        for X, y in dataloader: 
            X, y = X.to(device), y.to(device)
            yhat = model(X)
            loss = loss_fn(yhat, y) ## valid loss
            acc = accuracy(y.cpu().data.numpy(), yhat.cpu().data.numpy().argmax(-1))       
            valid_avg_acc += (acc * len(y) / len(dataloader.dataset)) ## 각 배치마다 정확도(정답 개수 / 전체 개수)
            valid_avg_loss += loss.item() / len(dataloader) ## 각 배치마다 평가 오차 누적    

    return valid_avg_loss, valid_avg_acc

In [67]:
net_acc = [] ## fold별 valid셋의 평균 정확도
net_loss = [] ## fold별 valid셋의 평균 손실

for i, (train_idx, valid_idx) in enumerate(skfold.split(train_images, target)):
    gc.collect()
    torch.cuda.empty_cache()
    print(f'{i + 1} Fold Training......')
    
    X_train, X_valid = train_images[train_idx], train_images[valid_idx] 
    y_train, y_valid = target.iloc[train_idx], target.iloc[valid_idx]
    y_train = torch.tensor(y_train.to_numpy(), dtype=torch.int64) ## target을 텐서로 변환
    y_valid = torch.tensor(y_valid.to_numpy(), dtype=torch.int64) ## target을 텐서로 변환
    
    ## early stopping
    early_stopping = EarlyStopping(patience=20,
                                   verbose=False,
                                   path=SAVE_PATH,
                                   n_fold=i+1) ## patience 횟수 에폭후에도 valid loss가 작아지지 않으면 조기 중단
    
    ## Linear 모델
    net = CNN().to(device)
    net.apply(init_weights) ## Linear layer 가중치 초기화
    
    ## Dataset, Dataloader
    train_dataset = CustomDataset(X_train, y_train, train_mode=True, transforms=get_transform(train_mode=False))
    valid_dataset = CustomDataset(X_valid, y_valid, train_mode=True, transforms=get_transform(train_mode=False))
    
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

    ## optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE) ## 옵티마이저에 최적화할 파라미터와 학습률 전달
    
    ## scheduler
    # scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
    #                                               lr_lambda=lambda epoch: 0.95 ** epoch,
    #                                               last_epoch=-1,
    #                                               verbose=False)
    
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-4, last_epoch=-1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=1e-4)
    
    ## fold별로 모델 학습
    for epoch in tqdm(range(EPOCHS)): ## (배치사이즈 * 에폭)만큼 훈련시킴
        net, train_avg_loss = train(net, train_dataloader, optimizer, loss_fn, scheduler)  ## 모델 학습
        valid_avg_loss, valid_avg_acc = evaluate(net, valid_dataloader, loss_fn)  ## 모델 평가

        if epoch % 10 == 0 or epoch == EPOCHS - 1: 
            ## 10의 배수값을 가지는 에폭마다 평균 배치 훈련 오차와 평가 오차 출력
            print('[Epoch: {:>3}] train loss = {:>.5}  valid loss = {:>.5}'.format(epoch + 1, train_avg_loss, valid_avg_loss)) 
            
        ## epoch마다 early stopping 실행
        early_stopping(net, valid_avg_loss, valid_avg_acc) ## __call__ function
        if early_stopping.early_stop: ## early_stop이 true이면
            if epoch % 10 != 0 and epoch != EPOCHS - 1:
                print('[Epoch: {:>3}] train loss = {:>.5}  valid loss = {:>.5}'.format(epoch + 1, train_avg_loss, valid_avg_loss)) 
            print('Early stopping!')
            break 

    net_acc.append(early_stopping.val_acc) ## fold별 loss가 가장 작은 모델의 정확도
    net_loss.append(early_stopping.val_loss_min) ## fold별 loss가 가장 작은 모델의 손실
    
    ## fold별 평가 루프 종료시 가장 작은 loss와 이때의 accuracy를 출력
    print(f'{i + 1} Fold -> Best Valid Loss: {early_stopping.val_loss_min:.4f}  Best Valid Accuracy: {early_stopping.val_acc:.4f}\n\n')
    
## 마지막으로 폴드별 가장 loss가 작은 모델들의 평균 정확도와 평균 손실을 출력
print(f'{skfold.n_splits}Fold Mean Valid Accuracy: {np.mean(net_acc):.4f}')
print(f'{skfold.n_splits}Fold Mean Valid Loss: {np.mean(net_loss):.4f}')     

1 Fold Training......


  0%|          | 1/300 [00:01<09:55,  1.99s/it]

[Epoch:   1] train loss = 2.4911  valid loss = 2.4102


  4%|▎         | 11/300 [00:19<08:08,  1.69s/it]

[Epoch:  11] train loss = 1.8218  valid loss = 1.9082


...

[Epoch: 111] train loss = 0.26516  valid loss = 0.36075


 40%|████      | 121/300 [03:22<04:53,  1.64s/it]

[Epoch: 121] train loss = 0.3857  valid loss = 0.76145


 40%|████      | 121/300 [03:24<05:02,  1.69s/it]

[Epoch: 122] train loss = 0.35182  valid loss = 0.86338
Early stopping!
10 Fold -> Best Valid Loss: 0.2638  Best Valid Accuracy: 0.9294


10Fold Mean Valid Accuracy: 0.7904
10Fold Mean Valid Loss: 0.5948


## test 예측

`-` softmax function을 취하면 11개의 원소 중 최대값의 인덱스 번호가 최종 예측값이 된다

In [68]:
def predict(model: nn.Module, dataloader, weight_save_path):
    """저장된 모델의 가중치를 불러와서 dataloader의 각 데이터를 예측하여 반환"""
    model = model.to(device)
    weight_path_list = glob(weight_save_path + '/*.pt')
    test_probs = np.zeros(shape=(len(dataloader.dataset), 11)) ## test예측값

    for weight in weight_path_list :
        model.load_state_dict(torch.load(weight))
        model.eval()
        probs = None
        
        with torch.no_grad(): 
            for test_batch in dataloader:
                test_batch = test_batch.to(device)
                outputs = model(test_batch).cpu().numpy()
                if probs is None:
                    probs = outputs
                else:
                    probs = np.concatenate([probs, outputs])

        test_probs += (probs / N_FOLD)

    _, test_preds = torch.max(torch.tensor(test_probs), dim=1) ## 최대값과 인덱스를 반환
    return test_preds  

In [69]:
net = CNN().to(device)

In [81]:
test_dataset = CustomDataset(test_images, label_list=None, train_mode=False, transforms=get_transform(train_mode=False))
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [82]:
pred = predict(net, test_dataloader, SAVE_PATH)

In [83]:
pred = label_encoding_inverse_transform(pd.Series(pred))

In [84]:
submission['label'] = pred
submission.head()

,file_name,label
0,001.png,1
1,002.png,6
2,003.png,1
3,004.png,6
4,005.png,8


In [85]:
submission.to_csv('./Data/submission.csv', index=False)